# B-Splines



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import BSpline, make_lsq_spline

In [ ]:
# Basis function using BSpline
def B(x, k, i, t):
    c = np.zeros(len(t) - k - 1)
    c[i] = 1
    return BSpline(t, c, k)(x)

In [ ]:
# Degree of the spline
k = 3
rng = np.random.default_rng(787)
# Sample non-linear data
x_data = np.linspace(0, 5, 15)
y_data = np.sin(x_data) + 0.1 * rng.integers(low=0, high=10, size=len(x_data))
plt.plot(x_data, y_data, 'o', label='Data Points')
plt.grid(True)
plt.show()

In [ ]:
n_internal_knots = 5

# t_internal = np.quantile(x_data, np.linspace(0, 1, n_knots))
t_internal = np.quantile(x_data, np.linspace(0, 1, n_internal_knots + 2)[1:-1])

t = np.concatenate((
    [x_data[0]] * (k + 1),
    t_internal,
    [x_data[-1]] * (k + 1)
))
t

In [ ]:
for knot in t:
    plt.axvline(knot, color='gray', linestyle='--', alpha=0.5)
plt.plot(x_data, y_data, 'o', label='Data Points')

In [ ]:
# Fit B-spline to data
spline = make_lsq_spline(x_data, y_data, t, k)

# Dense evaluation grid
x_dense = np.linspace(x_data[0], x_data[-1], 1000)
y_dense = spline(x_dense)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x_data, y_data, 'o', label='Data Points')
plt.plot(x_dense, y_dense, 'r-', label='Fitted B-Spline')

for knot in t:
    plt.axvline(knot, color='gray', linestyle='--', alpha=0.5)
plt.plot(t, spline(t), 'ko', label='Knots')

plt.title('B-Spline Fit with Knots and Non-linear Data')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
n_basis = len(t) - k - 1
for i in range(n_basis):
    y = np.array([B(xi, k, i, t) for xi in x_dense])
    plt.plot(x_dense, y, label=f'$B_{{{i},{k}}}(x)$')

for knot in t:
    plt.axvline(knot, color='gray', linestyle='--', alpha=0.3)

plt.title(f'B-Spline Basis Functions (k = {k})')
plt.xlabel('x')
plt.ylabel('Basis Function Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
x_start = t[k]      # start of valid domain
x_end   = t[k+1]    # end of first segment
x_segment = np.linspace(t[k], t[k+1], 200)


In [ ]:
from scipy.interpolate import BSpline

basis_values = []

for i in range(len(spline.c)):
    # Create a basis spline with only 1 coefficient set to 1
    coeffs = np.zeros_like(spline.c)
    coeffs[i] = 1
    B_i = BSpline(t, coeffs, k)
    basis_values.append(B_i(x_segment))
segment_sum = np.zeros_like(x_segment)
for i in range(len(spline.c)):
    segment_sum += spline.c[i] * basis_values[i]


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(x_segment, segment_sum, label='First Segment Sum (Manual)', color='red')
plt.plot(x_segment, spline(x_segment), '--', label='Spline.eval()', color='blue')
plt.legend()
plt.title("First Segment of Spline = Sum of Weighted Basis Functions")
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from patsy import dmatrix
from mpl_toolkits.mplot3d import Axes3D  # Needed for 3D plotting
import seaborn as sns

# Seaborn style
sns.set_style("whitegrid")
sns.set_palette("coolwarm")

# Create grid
x1 = np.linspace(0., 1., 100)
x2 = np.linspace(0., 1., 100)
x1, x2 = np.meshgrid(x1, x2)

df = 3

# Tensor product basis using patsy
y = dmatrix("te(cr(x1, df), cc(x2, df)) - 1",
            {"x1": x1.ravel(), "x2": x2.ravel(), "df": df})

print(y.shape)  # (10000, 9)

# Plot
fig = plt.figure(figsize=(12, 12))
fig.suptitle("Tensor product basis example (2 covariates)", fontsize=16)

for i in range(df * df):
    ax = fig.add_subplot(df, df, i + 1, projection='3d')
    yi = y[:, i].reshape(x1.shape)
    
    # Surface plot
    ax.plot_surface(x1, x2, yi, rstride=4, cstride=4, alpha=0.15, cmap=cm.coolwarm)
    
    # Contours
    ax.contour(x1, x2, yi, zdir='z', cmap=cm.coolwarm, offset=-0.5)
    ax.contour(x1, x2, yi, zdir='y', cmap=cm.coolwarm, offset=1.2)
    ax.contour(x1, x2, yi, zdir='x', cmap=cm.coolwarm, offset=-0.2)
    
    # Axes limits
    ax.set_xlim3d(-0.2, 1.0)
    ax.set_ylim3d(0, 1.2)
    ax.set_zlim3d(-0.5, 1)
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_zticks([-0.5, 0, 1])

fig.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from patsy import dmatrix
from mpl_toolkits.mplot3d import Axes3D  # Needed for 3D plotting
import seaborn as sns

# Seaborn style for axes/grid
sns.set_style("whitegrid")
sns.set_palette("coolwarm")

# Create grid
x1 = np.linspace(0., 1., 100)
x2 = np.linspace(0., 1., 100)
x1, x2 = np.meshgrid(x1, x2)

df = 3

# Tensor product basis using patsy
y = dmatrix("te(cr(x1, df), cc(x2, df)) - 1",
            {"x1": x1.ravel(), "x2": x2.ravel(), "df": df})

print(y.shape)  # (10000, 9)

# Plot
fig = plt.figure(figsize=(12, 12))
fig.suptitle("Tensor product basis example (2 covariates)", fontsize=16)

for i in range(df * df):
    ax = fig.add_subplot(df, df, i + 1, projection='3d')
    yi = y[:, i].reshape(x1.shape)
    
    # Surface plot with viridis colormap and black mesh lines
    ax.plot_surface(x1, x2, yi, rstride=4, cstride=4, alpha=0.8, cmap='viridis', edgecolor='k')
    
    # Contours
    ax.contour(x1, x2, yi, zdir='z', cmap='viridis', offset=-0.5)
    ax.contour(x1, x2, yi, zdir='y', cmap='viridis', offset=1.2)
    ax.contour(x1, x2, yi, zdir='x', cmap='viridis', offset=-0.2)
    
    # Axes limits
    ax.set_xlim3d(-0.2, 1.0)
    ax.set_ylim3d(0, 1.2)
    ax.set_zlim3d(-0.5, 1)
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_zticks([-0.5, 0, 1])

fig.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
